In [1]:
fast.two.sum <- function(a,b) {                   # known abs(a) >= abs(b)
  x <- a+b
  bv <- x-a
  y <- b-bv
  if(y==0) return(x)
  c(x,y)
}

two.sum <- function(a,b) {                        # unknown order
  x <- a+b
  bv <- x-a
  av <- x-bv
  br <- b-bv
  ar <- a-av
  y <- ar+br
  if(y==0) return(x)
  c(x,y)
}

expansion.sum <- function(g) {
  g <- g[order(abs(g))]
  z <- fast.two.sum(g[2],g[1])
  q <- z[1]
  h <- NULL
  if(length(z)!=1) h <- z[2]
  n <- length(g)
  if(n==2) return(c(h,q))
  for(i in 3:n) {
    z <- two.sum(q,g[i])
    q <- z[1]
    if(length(z)!=1) h <- c(h,z[2])
  }
  c(h,q)                                          # strongly non-overlapping values
}

In [2]:
hlm <- function(z) {
  zz <- outer(z,z,"+")
  zz <- zz[lower.tri(zz,diag=TRUE)]
  median(zz, na.rm=TRUE)/2			              ###v3.1 ignore missing values
}

In [3]:
JTK.AMPFACTOR <- sqrt(2)                          # 1/median(abs(cosine)) used to calculate amplitudes
JTK.PIHAT <- round(pi,4)                          # replacement for pi to ensure unique cos values
JTK.ALT <<- list()				                  ###v3.1 container for alternative distributions

# jtkdist: calculate the exact null distribution using the Harding algorithm
# http://www.jstor.org/pss/2347656
###v3.1 modified to provide alternative distributions for data with missing values
jtkdist <- function(timepoints,reps=1,normal=FALSE,alt=FALSE) {
  
  if(length(reps)==timepoints) {
    tim <- reps                                   # support for unbalanced replication
  } else {
    tim <- rep(reps[1],timepoints)                # balanced replication
  }
  
  maxnlp <- lfactorial(sum(tim))-sum(lfactorial(tim)) #maximum possible negative log p-value
  limit <- log(.Machine$double.xmax)                  #largest representable nlp
  normal <- normal | (maxnlp>limit-1)                 #switch to normal approximation if maxnlp is too large
  
  if(alt) {
    lab <- paste(sort(tim), collapse=",")
    if(lab %in% names(JTK.ALT)) {
      alt.id <- match(lab, names(JTK.ALT))
      return(alt.id)
    }
    nn <- sum(tim)
    M <- (nn^2-sum(tim^2))/2
    JTK.ALT[[lab]] <<- list()
    JTK.ALT[[lab]]$MAX <<- M
    if(normal) {
      var <- (nn^2*(2*nn+3) - 
	      sum(tim^2*(2*tim+3)))/72
      JTK.ALT[[lab]]$SDV <<- sqrt(var)
      JTK.ALT[[lab]]$EXV <<- M/2
      return(length(JTK.ALT))
    }
  } else {
    JTK.GRP.SIZE <<- tim                          # sizes of each replicate group
    JTK.NUM.GRPS <<- length(tim)                  # timepoints = number of groups
    JTK.NUM.VALS <<- nn <- sum(tim)               # number of data values (independent of period and lag)
    JTK.MAX <<- M <- (nn^2-sum(tim^2))/2          # maximum possible jtk statistic
    JTK.GRPS <<- rep(1:length(tim), ti=tim)	      ### group labels
    JTK.DIMS <<- c(nn*(nn-1)/2,1)

    if(normal) {
      JTK.VAR <<- (nn^2*(2*nn+3) - 
		    sum(tim^2*(2*tim+3)))/72              # variance of jtk
      JTK.SDV <<- sqrt(JTK.VAR)                   # standard deviation of jtk
      JTK.EXV <<- JTK.MAX/2                       # expected value of jtk
      JTK.EXACT <<- FALSE
      return(invisible(0))                        # omit calculation of exact distribution
    }
  }
  MM <- floor(M/2)                          	  ### mode of this possibly alternative jtk distribution
  cf <- as.list(rep(1,MM+1))                      # initial lower half cumulative frequency distribution
    
  size <- tim                            	      ### sizes of each group of known replicate values
  size <- size[order(size)]                       # ascending order for fastest calculation
  k <- length(tim)                                ### number of groups of known replicate values
  
  N <- size[k]                  
  if(k>2) for(i in (k-1):2) {
    N <- c(size[i]+N[1],N)
  }
  for(i in 1:(k-1)) {                             # count permutations using the Harding algorithm
    m <- size[i]
    n <- N[i]
    
    if(n < MM) {
      P <- min(m+n,MM)
      for(t in (n+1):P) {                         # zero-based offset t
        for(u in 1+MM:t) {                        # one-based descending index u
          cf[[u]] <- expansion.sum(               # Shewchuck algorithm
            c(cf[[u]],-cf[[u-t]]))
        }
      }
    }
    Q <- min(m,MM)
    for(s in 1:Q) {                               # zero-based offset s
      for(u in 1+s:MM) {                          # one-based ascending index u
        cf[[u]] <- expansion.sum(                 # Shewchuck algorithm
          c(cf[[u]],cf[[u-s]]))
      }
    }
  }
  cf <- sapply(cf,sum)
  # cf now contains the lower-half cumulative frequency distribution;
  # append the symmetric upper-half cumulative distribution to cf
  
  if(M %% 2) {
    cf <- c(cf,2*cf[MM+1]-c(cf[MM:1],0))          # if M is odd (mode is duplicated)
  } else {
    cf <- c(cf,cf[MM+1]+cf[MM]-c(cf[MM:2-1],0))   # if M is even (unique mode is in lower half)
  }
  jtkcf <- rev(cf)                                # upper-tail cumulative frequencies for all integer jtk
  ajtkcf <- (jtkcf[-length(cf)]+jtkcf[-1])/2      # interpolated cumulative frequency values for all half-integer jtk  
  
  id <- 1+0:(2*M)                           	  ### one-based indices for all jtk values
  cf <- id                                        # container for the jtk frequency distribution
  cf[!!id%%2] <- jtkcf                            # odd indices for integer jtk
  cf[!id%%2] <- ajtkcf                            # even indices for half-integer jtk
  cp <- cf/jtkcf[1]                               # all upper-tail p-values
  
  if(alt) {                                       
    JTK.ALT[[lab]]$CP <<- cp
    return(length(JTK.ALT))
  }
  JTK.CP <<- cp
  JTK.EXACT <<- TRUE
}

In [4]:
jtk.init <- function(periods, interval=1) {
      
  JTK.INTERVAL <<- interval
  JTK.PERIODS <<- periods
  JTK.PERFACTOR <<- rep(1:length(periods),ti=periods)
  
  tim <- JTK.GRP.SIZE
  timepoints <- JTK.NUM.GRPS
  timerange <- 1:timepoints-1                     # zero-based time indices
  JTK.CGOOSV <<- list()      
  JTK.SIGNCOS <<- list()

  for(i in 1:length(periods)) {
    period <- periods[i]
    time2angle <- 2*JTK.PIHAT/period              # convert time to angle using an approximate pi value
    theta <- timerange*time2angle                 # zero-based angular values across time indices
    cos.v <- cos(theta)                           # unique cosine values at each timepoint
    cos.r <- rank(cos.v)                          # ranks of unique cosine values
    cos.r <- rep(cos.r,ti=tim)                    # replicated ranks
  
    cgoos <- sign(outer(cos.r,cos.r,"-"))
    cgoos <- cgoos[lower.tri(cgoos)]
    cgoosv <- array(cgoos,dim=JTK.DIMS)
    JTK.CGOOSV[[i]] <<- matrix(
      ncol=period,nrow=nrow(cgoosv)
    )
    JTK.CGOOSV[[i]][,1] <<- cgoosv
  
    cycles <- floor(timepoints/period)		      # v2.1
    range <- 1:(cycles*period)			          # v2.1
    cos.s <- sign(cos.v)[range]                   # signs over all full cycles (v2.1)            
    cos.s <- rep(cos.s,ti=tim[range])
    JTK.SIGNCOS[[i]] <<- matrix(
      ncol=period,nrow=length(cos.s)
    )
    JTK.SIGNCOS[[i]][,1] <<- cos.s
    
    for(j in 2:period) {                          # one-based half-integer lag index j
      delta.theta <- (j-1)*time2angle/2           # angles of half-integer lags
      cos.v <- cos(theta+delta.theta)             # cycle left
      cos.r <- rank(cos.v)                        # ranks of unique phase-shifted cosine values
      cos.r <- rep(cos.r,ti=tim)                  # phase-shifted replicated ranks
    
      cgoos <- sign(outer(cos.r,cos.r,"-"))
      cgoos <- cgoos[lower.tri(cgoos)]
      cgoosv <- array(cgoos,dim=JTK.DIMS)
      JTK.CGOOSV[[i]][,j] <<- cgoosv
    
      cos.s <- sign(cos.v)[range]
      cos.s <- rep(cos.s,ti=tim[range])
      JTK.SIGNCOS[[i]][,j] <<- cos.s    
    }
  }
}

In [5]:
jtkstat <- function(z) {
  alt <- any(is.na(z))				              ### flag for handling missing values
  if(alt) {
    tab <- table(JTK.GRPS[is.finite(z)])
    alt.id <- jtkdist(length(tab),
		      as.integer(tab),
		      alt=alt)
  }
  M <- switch(1+alt,			  	              ### maximum possible S score for this distribution
	      JTK.MAX,
	      JTK.ALT[[alt.id]]$MAX)
	      
  foosv <- sign(outer(z,z,"-"))
  foosv <- foosv[lower.tri(foosv)]
  dim(foosv) <- JTK.DIMS
  
  JTK.CJTK <<- list()                             
  for(i in 1:length(JTK.PERIODS)) {
    JTK.CJTK[[i]] <<- apply(JTK.CGOOSV[[i]],2,
      function(cgoosv) {
        S <- sum(foosv*cgoosv, na.rm=TRUE)        ### Kendall's S score ignoring missing values
        if(!S) return(c(1,0,0))
        jtk <- (abs(S)+M)/2                       ### two-tailed JTK statistic for this lag and distribution
        if(JTK.EXACT) {
          jtki <- 1+2*jtk                         # index into the exact upper-tail distribution
          p <- switch(1+alt,			 
		      2*JTK.CP[jtki],
		      2*JTK.ALT[[alt.id]]$CP[jtki])
        } else {
          p <- switch(1+alt,			 
		      2*pnorm(-(jtk-1/2),
			-JTK.EXV,JTK.SDV),
		      2*pnorm(-(jtk-1/2),
			-JTK.ALT[[alt.id]]$EXV,
			JTK.ALT[[alt.id]]$SDV))
        }
        c(p,S,S/M)				                  ### include tau = S/M for this lag and distribution
    })
  }
}

In [6]:
jtkx <- function(z, ampci=FALSE, conf=0.8) {      ###v3.1 'ampci=TRUE' for calculating amplitude confidence
  
  jtkstat(z)                                      # calculate p and S for all (period,phase) combos
  pvals <- lapply(JTK.CJTK,function(cjtk) {
    return(cjtk[1,])
  })
  
  # exact two-tailed p-values for all (period,phase) combos

  padj <- p.adjust(unlist(pvals),"bonf")          # Bonferroni adjusted two-tailed p-values
  JTK.ADJP <<- min(padj)                          # global minimum adjusted p-value
  
  padj <- split(padj,JTK.PERFACTOR)
  minpadj <- sapply(padj,min)                     # minimum adjusted p-value for each period
  
  peris <- which(JTK.ADJP==minpadj)               # indices of all optimal periods
  pers <- JTK.PERIODS[peris]                      # all optimal periods
  
  lagis <- lapply(padj[peris],function(z) {
    which(JTK.ADJP==z)
  })# list of optimal lag indices for each optimal period
  
  
  count <- sum(sapply(lagis,length))              # total number of optimal lags for all optimal period
  
  bestper <- 0
  bestlag <- 0
  besttau <- 0                                    
  maxamp <- 0
  maxamp.ci <- numeric(2)                        
  maxamp.pval <- 0                              
  
  for(i in 1:length(pers)) {
    per <- pers[i]
    peri <- peris[i]
    cjtk <- JTK.CJTK[[peri]]
    sc <- JTK.SIGNCOS[[peri]]
    w <- z[1:nrow(sc)]		    		  
    w <- (w-hlm(w))*JTK.AMPFACTOR	    	  
    
    for(lagi in lagis[[i]]) {  
      S <- cjtk[2,lagi]                           # optimal Kendall's S
      s <- sign(S)
      if(!s) s <- 1
  
      lag <- (per +(1-s)*per/4 -(lagi-1)/2)%%per
      signcos <- sc[,lagi]
      tmp <- s*w*signcos			             
      amp <- hlm(tmp)				              ###v3.1 allows missing values
      if (ampci)                                  ###v3.1 the calculation of amplitude confidence is optimal
	  {
		wt <- wilcox.test(tmp[is.finite(tmp)], 
		      conf.int=TRUE, conf.level=conf, exact=FALSE)
	    amp <- as.numeric(wt$estimate)
	  }
      if(amp > maxamp) {
	      maxamp <- amp
	      bestper <- per
	      bestlag <- lag
	      besttau <- abs(cjtk[3,lagi])            ###v3.1
		  if (ampci)                              ###v3.1
	      {
			maxamp.ci <- as.numeric(wt$conf.int)   
			maxamp.pval <- as.numeric(wt$p.value)
		  }
      }
    }
  }
  JTK.PERIOD <<- JTK.INTERVAL*bestper        	  # period (hours) with max amp
  JTK.LAG <<- JTK.INTERVAL*bestlag           	  # lag (hours) to peak with max amp
  JTK.AMP <<- max(0,maxamp)                	      # max amp
  JTK.TAU <<- besttau                             ### v3.1
  JTK.AMP.CI <<- maxamp.ci				          ### confidence interval for max amp; 'JTK.AMP.CI' is 'c(0,0)' if 'ampci=FALSE'
  JTK.AMP.PVAL <<- maxamp.pval			          ### p-value for max amp; 'JTK.AMP.PVAL' is '0' if 'ampci=FALSE'            
}

In [13]:
# Test Adr
data <- read.csv('./results/STAR/WT_supp.csv')
annot <- data[,1]
data <- data[,2:49]
data

WT_00_rep2,WT_02_rep2,WT_04_rep2,WT_06_rep2,WT_08_rep2,WT_10_rep2,WT_12_rep2,WT_14_rep2,WT_16_rep2,WT_18_rep2,...,WT_28_rep1,WT_30_rep1,WT_32_rep1,WT_34_rep1,WT_36_rep1,WT_38_rep1,WT_40_rep1,WT_42_rep1,WT_44_rep1,WT_46_rep1
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.93157116,7.65607590,3.86342604,7.54212411,6.2012463,6.1816939,6.1698241,8.09590007,4.31845385,5.4680030,...,3.8404962,4.35511837,4.29096131,4.884628e+00,8.7999677,6.3138077,5.88698161,11340.949429,7.6302217,7.0456442
0.27520998,0.08902414,0.15453704,0.17957438,0.0000000,0.1287853,0.1801408,0.08995445,0.04852195,0.3417502,...,0.0738557,0.00000000,0.09752185,5.367723e-02,3.1145953,0.7731193,0.05660559,8615.711097,1.7074622,0.3945561
36.01319184,99.57349871,23.10328769,20.47147973,20.1179967,104.1229065,73.5875370,367.50888615,27.90012319,28.9348490,...,212.4828387,35.40900589,243.85338078,1.230175e+03,64.1211129,356.6012825,63.28505235,89.111330,44.2873010,876.1963081
70.09991380,49.49742090,13.67652816,37.44125897,52.2058410,140.0540022,295.4760283,231.81260546,104.07958998,193.4306045,...,44.9042634,59.45683342,158.18043736,2.011822e+02,138.4264580,162.8704683,135.51378830,33.791016,93.7503468,146.6621289
58.69835744,101.08690906,105.31699372,100.65144199,66.7715591,20.7988242,108.3547209,91.57362528,77.63512539,223.5046208,...,89.1438256,43.67741903,90.01266565,1.235650e+02,101.4962565,82.5304865,52.52998978,170.309573,114.5066843,132.0071890
0.07863142,0.08902414,0.07726852,0.08978719,0.0000000,0.0000000,0.2251761,0.22488611,0.09704391,0.1139167,...,0.0738557,0.06311766,0.24380462,5.367723e-02,0.9887604,0.0000000,0.00000000,4624.735437,0.9070893,0.2818258
19.61854009,30.49076737,15.45370414,8.52978322,7.4270741,44.1733543,26.3005640,181.52807055,15.62406899,10.5372974,...,164.7720588,13.94900232,101.42272186,1.013372e+03,33.1729119,171.6969138,25.07627745,17.822266,11.7388027,497.8170340
72.26227794,62.80652958,62.58750177,58.72082343,55.9554318,210.1775923,137.7176794,132.41294344,103.35176068,114.9419788,...,79.2471623,68.92448206,116.87993476,1.522286e+02,51.6132936,78.9870230,99.51263151,13.331055,82.4917679,172.6464646
207.94079872,262.75474424,113.58472544,55.21912295,42.3992189,15.9049832,16.9332398,40.02972815,90.10526741,113.9736866,...,119.4985169,37.36565328,22.57630780,1.390240e+01,31.3931432,66.4238342,100.64474336,55.819337,107.6234773,114.0267051


In [14]:
jtkdist(timepoints = 48)
periods <- 6:12
jtk.init(periods,2)

In [15]:
cat("JTK analysis started on",date(),"\n")
flush.console()

JTK analysis started on Wed Jun 14 14:31:25 2023 


In [16]:
st <- system.time({
  res <- apply(data,1,function(z) {
	jtkx(z)
  c(JTK.ADJP,JTK.PERIOD,JTK.LAG,JTK.AMP)
  })
  res <- as.data.frame(t(res))
  bhq <- p.adjust(unlist(res[,1]),"BH")
  res <- cbind(bhq,res)
  colnames(res) <- c("BH.Q","ADJ.P","PER","LAG","AMP")
  results <- cbind(annot,res)
})
print(st)

   user  system elapsed 
  7.004   0.017   7.022 


In [17]:
results

annot,BH.Q,ADJ.P,PER,LAG,AMP
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Gpx3,4.184819e-01,3.520689e-01,24,13,0.70111867
Kap,1.572701e-02,8.532579e-03,24,19,0.12837315
Saa1,6.563608e-01,5.785513e-01,16,15,22.86925637
Cyp7a1,1.293189e-03,4.877277e-04,24,15,55.66889718
Hba-a1,2.458321e-01,1.961252e-01,20,0,19.19039921
Slc34a1,1.000000e+00,1.000000e+00,24,18,0.08986775
Saa2,5.940141e-01,5.193364e-01,16,15,11.19300806
Hhex,5.123023e-01,4.405506e-01,22,15,16.37776746
Tubb2a,8.827767e-15,6.329661e-17,24,23,69.40856406


In [18]:
write.csv(results,'./JTK_results/JTK_WT_supp.csv')